# TD2 part 2: Named entity recognition

Dans ce TD, nous allons prendre un datasets où les noms de personnes sont taggés.<br>
Nous allons transformer ces données en tenseurs X, y et attention_mask.<br>
Nous allons créer un modèle RNN pour prédire si un mot est un nom de personne.<br>
Nous allons ensuite créer la loop avec l'optimizer pour apprendre le modèle.<br>
Du modèle appris (prédisant sur les tokens), nous allons postprocess les prédictions pour avoir les prédictions sur les noms.

Un fois que la loop est créée et que le modèle apprend, nous allons changer la structure du modèle:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.

Lorsqu'on a un bon modèle de prédiction pour les noms de personnes, nous allons l'appliquer à notre projet fil rouge.
Utilisez-le tel que. Quelle accuracy ?
Ré-entrainez la (les) dernière(s) couche(s) du modèle sur notre jeu de données. A-t-il gagné en accuracy ?

In [1]:
# Import
import matplotlib.pyplot as plt
import numpy as np
import torch
import transformers

## Data

Télécharger le dataset MultiNERD FR [ici](https://github.com/Babelscape/multinerd)<br>
Mettez les données dans le dossier data/raw du projet.


In [5]:
def extract_multinerd_person_words(filename="../data/raw/train_fr.tsv"):
    with open(filename, encoding='utf8') as f:
        tagged_words = [line.strip().split("\t") for line in f]
        
        # Joining words until we meet a dot
        # Word's label is 1 if 'PER' is in its tag
        sentences = []
        sentence_labels = []
    
        this_word = []
        this_labels = []
        for tagged_word in tagged_words:
            if len(tagged_word) < 3:
                # not a tagged word
                continue
            word = tagged_word[1]
            tag = tagged_word[2]
        
            if word == '.':
                sentences.append(" ".join(this_word))
                sentence_labels.append(this_labels)
            
                this_word = []
                this_labels = []
            else:
                this_word.append(word)
                this_labels.append(1 * tag.endswith("PER"))

    return sentences, sentence_labels


In [6]:
sentences, labels = extract_multinerd_person_words("../data/raw/train_fr.tsv")

## Tokenizer

En utilisant le tokenizer d'HuggingFace "camembert-base":
- Transformer les phrases en tokens
- Obtenez des vecteur y qui ont le même nombre d'entrées qu'il y a de tokens dans la phrase
- Ayez un tenseur "attention_mask" pour savoir sur quels tokens on cherche à predire le label
- Transformez les tokens en token_ids (avec le tokenizer)
Avec tout cela, vous pouvez former vos tenseurs X, Y et attention_mask

In [11]:
from transformers import AutoTokenizer, CamembertTokenizer, CamembertForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = CamembertForTokenClassification.from_pretrained("camembert-base")

# Exemple de phrase
sentence = "Votre phrase ici."

# Tokenizer la phrase
tokens = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

# Récupérer les tensor X, Y, et attention_mask
input_ids = tokens["input_ids"]
attention_mask = tokens["attention_mask"]

# Créer un tensor Y (étiquettes initiales, à adapter selon votre structure)
# Dans cet exemple, on suppose que chaque mot a une étiquette binaire (0 ou 1)
# Vous devrez adapter cela en fonction de la structure réelle de vos étiquettes
# Le nombre d'éléments dans Y doit correspondre au nombre de tokens dans la phrase
labels = torch.randint(2, (1, input_ids.size(1)))

# Afficher les tensors
print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)
print("Labels:", labels)

# Maintenant, vous pouvez utiliser ces tensors pour entraîner votre modèle


c:\users\daoud\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Daoud\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are

Input IDs: tensor([[   5, 1268, 3572,  323,    9,    6]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1]])
Labels: tensor([[0, 1, 1, 1, 0, 1]])


In [12]:
def build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels):
    words = sentence.split()
    tokens = []
    tokens_label = []
    attention_mask = []
    
    for word, label in zip(words, labels):
        this_tokens = tokenizer.tokenize(word)
        tokens += this_tokens
        
        this_labels = [0] * len(this_tokens)
        this_labels[0] = label        
        tokens_label += this_labels
        
        this_attention_mask = [1] + [0] * (len(this_tokens) - 1)
        attention_mask += this_attention_mask
        
    return tokens, tokens_label, attention_mask

In [13]:
tokens, label, padding_masks = build_tokens_and_labels_and_attention_mask(tokenizer, sentences[0], labels[0])

In [14]:
tokens

['▁Il', '▁est', '▁incarné', '▁par', '▁A', 'ustin', '▁S', 'to', 'well']

In [15]:
label

[tensor(0), tensor(1), tensor(1), tensor(1), tensor(0), 0, tensor(1), 0, 0]

In [16]:
padding_masks

[1, 1, 1, 1, 1, 0, 1, 0, 0]

## Model

Contruisez un modèle RNN comme dans la partie 1. Pour l'instant, il prendra comme arguments:
- Vocab size: le nombre de différents tokens du tokenizer (52 000 pour camembert-base)
- Embedding dim: la dimension de l'embedding des tokens (par défaut 50)
- hidden_dim: la dimension de l'état récurrent de votre RNN (par défaut 20)
- tagset_size: la nombre de classes possibles pour les prédictions (ici 2)

Dans le forward, votre modèle enchaînera les couches suivantes:
- Un embedding
- Un RNN
- Un ReLU
- Une couche linéaire
- Un softmax pour que la somme des prédictions pour une entrée soit égale à 1 (la prédiction pour un élément et sa probabilité d'être dans chaque classe)

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_dim=20, tagset_size=2):
        super(SimpleRNN, self).__init__()

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # RNN layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        # ReLU activation
        self.relu = nn.ReLU()

        # Linear layer
        self.linear = nn.Linear(hidden_dim, tagset_size)

        # Softmax activation
        self.softmax = nn.Softmax(dim=2)  # Softmax along the last dimension (tokens)

    def forward(self, x):
        # Embedding layer
        embedded = self.embedding(x)

        # RNN layer
        rnn_output, _ = self.rnn(embedded)

        # ReLU activation
        relu_output = self.relu(rnn_output)

        # Linear layer
        linear_output = self.linear(relu_output)

        # Softmax activation
        output = self.softmax(linear_output)

        return output

# Exemple d'utilisation du modèle avec un vocabulaire de taille 52000
vocab_size = 52000
embedding_dim = 50
hidden_dim = 20
tagset_size = 2

# Instanciation du modèle
model = SimpleRNN(vocab_size, embedding_dim, hidden_dim, tagset_size)

# Exemple d'entrée (batch de séquences de tokens)
example_input = torch.randint(vocab_size, (3, 10))  # Batch size: 3, Sequence length: 10

# Passe avant
output = model(example_input)

# Affichage de la sortie
print("Output shape:", output.shape)


Output shape: torch.Size([3, 10, 2])


## Optimizer

Je fournis ici une fonction prenant un modèle, des tenseurs X, y, attention_mask.
Pour chaque batch:
- La loop utilise le modèle pour prédire sur x_batch
- Avec attention_mask, elle identifie sur quels tokens les prédictions compte
- Elle regarde la cross entropy entre y\[attention_ids\] et yhat\[attention_ids\]
- Elle output un dictionnaire avec le model et la loss au fur et à mesure des itérations

Entraînez le modèle avec vos données. <br>
Plottez la loss history.<br>
Itérez sur le modèle pour l'améliorer:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.



In [23]:
def train_model(model, X, y, attention_masks, n_epochs=100, lr=0.05, batch_size=128):
    loss_function = torch.nn.CrossEntropyLoss()
    loss_history = []

    dataset = torch.utils.data.TensorDataset(X, y, attention_masks)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for i, (x_batch, y_batch, mask) in enumerate(loader):
            optimizer.zero_grad()
        
            ids = mask.reshape(-1)
            yhat = model(x_batch).reshape((-1, 2))[ids]
            this_y = y_batch.reshape(-1)[ids]
            
            loss = loss_function(yhat, this_y)
            loss.backward()
            
            loss_history.append(loss.clone().detach())
        
            optimizer.step()
        if epoch % 10 == 0:
            print(f"Got loss at {epoch}", np.mean(loss_history[-10:]))
    
    return {"model": model, "loss_history": loss_history}

In [24]:
# Supposons que vous ayez déjà défini le modèle (SimpleRNN), les données (X, y, attention_masks) et le tokenizer
# Instanciation du modèle
model = SimpleRNN(vocab_size, embedding_dim, hidden_dim, tagset_size)

# Tokenizer vos données
tokenized_data = tokenizer(sentences, return_tensors="pt", truncation=True, padding=True)

# Récupérer les tensors X, y, et attention_mask à partir du tokenized_data
X = tokenized_data["input_ids"]
y = torch.tensor(labels)
attention_masks = tokenized_data["attention_mask"]

# Afficher les dimensions des tensors
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Attention Masks shape:", attention_masks.shape)

# Ajuster les dimensions de y
max_sequence_length = 350
y = F.pad(y, pad=(0, max_sequence_length - y.size(1)), value=0)
# Ajuster les dimensions de y pour correspondre à X et attention_masks
y = y.squeeze(dim=0)

# Afficher les dimensions après ajustement
print("Adjusted y shape:", y.shape)


# Entraîner le modèle avec la fonction train_model
result = train_model(model, X, y, attention_masks, n_epochs=100, lr=0.05, batch_size=128)

# Récupérer le modèle entraîné et l'historique de la perte
trained_model = result["model"]
loss_history = result["loss_history"]

# Plotter l'historique de la perte
import matplotlib.pyplot as plt

plt.plot(loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss History')
plt.show()

# Itérer sur le modèle pour l'améliorer
# Par exemple, ajustez le learning rate, ajoutez des couches, etc.

# Exemple d'ajustement du learning rate et poursuite de l'entraînement
result = train_model(trained_model, X, y, attention_masks, n_epochs=50, lr=0.01, batch_size=128)

# Récupérer le modèle mis à jour et l'historique de la perte
updated_model = result["model"]
updated_loss_history = result["loss_history"]

# Plotter l'historique de la perte mis à jour
plt.plot(updated_loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Updated Training Loss History')
plt.show()


X shape: torch.Size([140436, 350])
y shape: torch.Size([1, 6])
Attention Masks shape: torch.Size([140436, 350])
Adjusted y shape: torch.Size([350])


C:\Users\Daoud\AppData\Local\Temp\ipykernel_20212\2713239486.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(labels)


AssertionError: Size mismatch between tensors

## Postprocessing

Créer une fonction prenant les prédictions du modèle (au niveau token) et sort les prédictions au niveau mot.<br>
Par exemple, admettons que, pour un mot, la prédiction du 1er token est la seule qu'on considère.<br>
si la phrase est "Bonjour John", avec les tokens \["bon", "jour", "Jo", "hn"\] avec les predictions \[0.12, 0.65, 0.88, 0.45\]<br>
Je veux récupérer les prédictions "bonjour": 0.12, "John": 0.88

In [26]:
def aggregate_predictions_by_word(token_predictions, words):
    word_predictions = {}
    current_word = ""
    current_sum = 0.0
    count = 0

    for token_pred, word in zip(token_predictions, words):
        if word.startswith("##"):
            # Traitement des tokens qui font partie d'un mot (suite du mot)
            current_word += word[2:]
            current_sum += token_pred
            count += 1
        else:
            # Nouveau mot
            if count > 0:
                # Calculer la moyenne des prédictions pour le mot précédent
                avg_pred = current_sum / count
                word_predictions[current_word] = avg_pred

            # Réinitialiser pour le nouveau mot
            current_word = word
            current_sum = token_pred
            count = 1

    # Traitement du dernier mot
    if count > 0:
        avg_pred = current_sum / count
        word_predictions[current_word] = avg_pred

    return word_predictions

In [27]:
# Exemple d'utilisation
token_predictions = [0.12, 0.65, 0.88, 0.45]
words = ["bon", "jour", "Jo", "hn"]

word_predictions = aggregate_predictions_by_word(token_predictions, words)

# Afficher les prédictions par mot
for word, pred in word_predictions.items():
    print(f"{word}: {pred}")

bon: 0.12
jour: 0.65
Jo: 0.88
hn: 0.45
